#       InSPIRe Vaginal Resistome Jupyter Notebook (1): processing & normalization

**Author : Nassim Boutouchent** 

Hello! In this first notebook we process data prior to downstream analyses. 
It mainly focuses on (i) normalization of Antibiotic Resistance genes (ARGs) and Mobile Genetic Elements (MGE) frequency tables using the RPKM method ; and (ii) processing data for (correlation) sparCC analysis and vizualisation.

**Note**: *For microbial species we used the previously DESeq2 normilized frequency table for downstream analysis (see Baud et al., 2023,https://doi.org/10.1038/s41598-023-36126-z).*

**External Dataset cross-validation** 
**dataset information** :
Feehily, C., Crosby, D., Walsh, C.J. et al.
Shotgun sequencing of the vaginal microbiome reveals both a species and functional potential signature of preterm birth.
#npj Biofilms Microbiomes 6, 50 (2020). https://doi.org/10.1038/s41522-020-00162-8 \
Data has been uploaded from the European Nucleotide Archive (ENA) under the study accession number PRJEB34536.


## Libraries

In [1]:
import pandas as pd
import numpy as np

## Data load 

In [2]:
## InSPIRE ARGs MGEs counts table
arg_count_raw = pd.read_csv("../data/Raw/inspire_ARG_count_RF_Raw.csv", index_col= "SampleID")
mge_count_raw = pd.read_csv("../data/Raw/inspire_MGE_count_Raw.csv", index_col= "SampleID")

## External Dataset ARGs counts table
external_arg_count_raw = pd.read_csv("../data/Raw/ExtSet_ARGs_count_RF_Raw.csv", index_col="SampleID")

##ARGs from CARD db
card_arg_count_raw = pd.read_csv("../data/Raw/inspire_ARG_count_CARD_Raw.csv")

## load gene lengths for RPK Norm 
ResFinderdb_gene_lengths = pd.read_csv("../data/supp_materials/ARG_ResFinder_v4.3.3_lenght.csv")
MGE_Custommgedb_gene_lengths = pd.read_csv("../data/supp_materials/MGEs_FINAL_99perc_trim_length.csv")
CARDdb_gene_lengths = pd.read_csv("../data/supp_materials/ARG_CARD_v3.3.0_length.csv")

# InSPIRe final species table (containing the final set of samples)
taxonomy_rabun = pd.read_csv("../data/Norm/inspire_taxonomy_rabun_sp.csv", index_col=0)

## External dataset final species table (containing all samples)
external_taxonomy_rabun = pd.read_csv("../data/Norm/ExternalDataset_taxonomy_rabun_sp.csv", index_col=0)

In [3]:
#initial formating 
external_arg_count_raw.head()

,erm(B)_18_X66468,erm(B)_10_U86375,erm(X)_3_U21300,tet(W)_1_DQ060146,erm(A)_2_AF002716,mef(A)_2_U83667,tet(M)_5_U58985,lsa(C)_1_HM990671,msr(D)_2_AF274302,tet(M)_13_AM990992,cfxA3_1_AF472622,blaTEM-116_1_AY425988,tet(M)_4_X75073,tet(O/W)_5_AM889122,tet(Q)_4_Z21523
SampleID,,,,,,,,,,,,,,,
NMH78,0,0,0,0,0,0,105,73,0,0,0,0,0,0,0
NMH64,0,0,0,0,0,0,768,390,0,0,0,0,0,0,0
NMH50,0,0,0,0,0,0,524,247,0,0,0,0,0,0,0
NMH63,0,0,0,0,0,0,0,457,0,0,0,0,0,0,0
NMH11,0,0,0,0,0,0,0,549,0,0,62,0,0,0,81


## (1) RPKM Norm 

In [4]:
# RPKM Normalization function
def compute_rpkm(count_table, geneID_lenght, gene_col="Gene", length_col="Length"):
    """
    RPKM = (10⁹ x C) / (N x L)
    C = number of reads mapped to a gene
    N = total mapped reads in the sample 
    L = gene lenght in bp 
    """
    gene_length_dict = geneID_lenght.set_index(gene_col)[length_col].to_dict()
    count_table = count_table.astype(int)
    
    rpk_df = count_table.div([gene_length_dict[col] / 1000 for col in count_table.columns], axis=1)
    total_reads_per_sample = count_table.sum(axis=1)
    scaling_factor = total_reads_per_sample / 1e6

    rpkm_df = rpk_df.div(scaling_factor, axis=0)

    return rpkm_df

# Format table at gene level
# initial formating : gene_variant_accessionnumber  -> gene
def format_columns(rpkm_count_table, n=0): 
    rpkm_count_table_gene = rpkm_count_table.copy()
    rpkm_count_table_gene.columns = [col.split('_')[n] for col in rpkm_count_table_gene.columns]
    rpkm_count_table_gene = rpkm_count_table_gene.groupby(axis=1, level=0).sum()

    return rpkm_count_table_gene

### Antibiotic resistance genes

#### from RF DB

In [5]:
# initial formating RF
print(ResFinderdb_gene_lengths.head(2))
print(external_arg_count_raw.head(2))

                      Gene  Length
0      aac(6')-Ib_2_M23634     606
1  aac(6')-Ib11_1_AY136758     570
          erm(B)_18_X66468  erm(B)_10_U86375  erm(X)_3_U21300  \
SampleID                                                        
NMH78                    0                 0                0   
NMH64                    0                 0                0   

          tet(W)_1_DQ060146  erm(A)_2_AF002716  mef(A)_2_U83667  \
SampleID                                                          
NMH78                     0                  0                0   
NMH64                     0                  0                0   

          tet(M)_5_U58985  lsa(C)_1_HM990671  msr(D)_2_AF274302  \
SampleID                                                          
NMH78                 105                 73                  0   
NMH64                 768                390                  0   

          tet(M)_13_AM990992  cfxA3_1_AF472622  blaTEM-116_1_AY425988  \
SampleID               

In [6]:
# InSPIRe ARGs RPKM Norm 
arg_count_rpkm = compute_rpkm(arg_count_raw, ResFinderdb_gene_lengths, gene_col="Gene", length_col="Length")

# External Dataset ARGs RPKM Norm
external_arg_count_rpkm = compute_rpkm(external_arg_count_raw, ResFinderdb_gene_lengths, gene_col="Gene", length_col="Length")
external_arg_count_rpkm.head()

,erm(B)_18_X66468,erm(B)_10_U86375,erm(X)_3_U21300,tet(W)_1_DQ060146,erm(A)_2_AF002716,mef(A)_2_U83667,tet(M)_5_U58985,lsa(C)_1_HM990671,msr(D)_2_AF274302,tet(M)_13_AM990992,cfxA3_1_AF472622,blaTEM-116_1_AY425988,tet(M)_4_X75073,tet(O/W)_5_AM889122,tet(Q)_4_Z21523
SampleID,,,,,,,,,,,,,,,
NMH78,0.0,0.0,0.0,0.0,0.0,0.0,307233.146067,277290.303956,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH64,0.0,0.0,0.0,0.0,0.0,0.0,345423.143351,227713.025535,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH50,0.0,0.0,0.0,0.0,0.0,0.0,353977.518374,216607.954511,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH63,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,676132.521974,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH11,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,536411.495035,0.0,0.0,92748.836152,0.0,0.0,0.0,60774.674518


In [7]:
# format columns to remove accession numbers and group by gene names 
arg_count_rpkm_gene = format_columns(arg_count_rpkm, n =0)
external_arg_count_rpkm_gene = format_columns(external_arg_count_rpkm, n =0)

print(arg_count_rpkm_gene.columns.tolist()[:10])
print(external_arg_count_rpkm_gene.columns.tolist()[:10])

['ARR-3', 'NarA', 'NarB', 'OqxA', 'OqxB', 'aac(3)-IIa', 'aac(3)-IId', 'aac(3)-IV', 'aac(3)-Id', 'aac(3)-XI']
['blaTEM-116', 'cfxA3', 'erm(A)', 'erm(B)', 'erm(X)', 'lsa(C)', 'mef(A)', 'msr(D)', 'tet(M)', 'tet(O/W)']


In [8]:
# print final formating 
arg_count_rpkm.head()

,ant(6)-Ia_5_AB247327,msr(A)_2_AB013298,aph(3'')-Ib_5_AF321551,blaZ_133_AIDT01000014,blaZ_117_ACHE01000004,blaZ_132_AHVO01000010,fusB_1_AY373761,cat(pC194)_1_NC_002013,sul3_2_AJ459418,sul1_36_X15024,...,vga(A)LC_2_DQ823382,dfrA1_9_AJ238350,blaLEN7_1_AJ635425,blaSHV-110_1_HQ877615,dfrA1_5_EU089668,tet(32)_2_EF626943,aadA1b_1_M95287,cfxA3_1_AF472622,blaTEM-182_1_HQ317449,msr(C)_2_AF313494
SampleID,,,,,,,,,,,,,,,,,,,,,
IMVM_0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Keep negative samples, as they have important biological meaning
inspire_samples = taxonomy_rabun.index.tolist()
arg_count_rpkm_gene = arg_count_rpkm_gene.reindex(inspire_samples, fill_value=0)

# Also keep negative samples for the External dataset
ExternalSet_samples = external_taxonomy_rabun.index.tolist()
external_arg_count_rpkm_gene = external_arg_count_rpkm_gene.reindex(ExternalSet_samples, fill_value=0)

# check 
print(arg_count_rpkm_gene.shape)
print(external_arg_count_rpkm_gene.shape)

#7 samples, from patients who withdrew their consent to the study, has been excluded for downstream analysis. 
#['IMVM_0217', 'IMVM_0712', 'IMVM_1200', 'IMVM_1304', 'IMVM_1607', 'IMVM_1794', 'IMVM_1841']

(1957, 231)
(89, 12)


#### from CARD  db

In [10]:
# initial formating CARD
print(card_arg_count_raw.head(2))
print(CARDdb_gene_lengths.head(2))

  Gene Name  Accession                                    AMR Gene Family  \
0      mdtA    3000792  resistance-nodulation-cell division (RND) anti...   
1      evgA    3000832  major facilitator superfamily (MFS) antibiotic...   

                                          Drug Class  IMVM_0001  IMVM_0002  \
0                           aminocoumarin antibiotic      920.0        0.0   
1  macrolide antibiotic; fluoroquinolone antibiot...      486.0        0.0   

   IMVM_0003  IMVM_0004  IMVM_0005  IMVM_0006  ...  IMVM_1975  IMVM_1976  \
0          0        0.0        0.0        0.0  ...        0.0        0.0   
1          0        0.0        0.0        0.0  ...        0.0        0.0   

   IMVM_1977  IMVM_1978  IMVM_1979  IMVM_1980  IMVM_1981  IMVM_1982  \
0        0.0        0.0        0.0     1036.0        0.0        0.0   
1        0.0        5.0        0.0      904.0        0.0        0.0   

   IMVM_1983  IMVM_1984  
0        0.0       22.0  
1        0.0        6.0  

[2 rows x 1

In [11]:
#format
card_arg_count_raw.rename(columns={"Accession": "ARO ID"}, inplace=True)
col_to_drop = ['AMR Gene Family', 'Drug Class','Gene Name','Gene Name']
card_arg_count_raw_ARO = card_arg_count_raw.copy()
card_arg_count_raw_ARO = card_arg_count_raw_ARO.set_index("ARO ID")
card_arg_count_raw_ARO = card_arg_count_raw_ARO.drop(columns=col_to_drop).T


In [12]:
# InSPIRe ARGs RPKM Norm 
card_arg_count_rpkm_ARO = compute_rpkm(card_arg_count_raw_ARO, CARDdb_gene_lengths, gene_col="ARO ID", length_col="Gene Length")

In [13]:
# Format to gene count table
ARO_to_gene = CARDdb_gene_lengths.set_index('ARO ID')['Gene Name'].to_dict()
card_arg_count_rpkm_gene = card_arg_count_rpkm_ARO.rename(columns=ARO_to_gene)
print(card_arg_count_rpkm_gene.columns.tolist()[:10])

['mdtA', 'evgA', 'mdtE', 'lsaA', 'emrB', 'tet(X)', 'H-NS', 'CRP', 'emrY', 'mel']


In [14]:
inspire_samples = taxonomy_rabun.index.tolist()
card_arg_count_rpkm_gene = card_arg_count_rpkm_gene.reindex(inspire_samples, fill_value=0).fillna(0)
print(card_arg_count_rpkm_gene.shape)

(1957, 492)


### MGEs 

In [15]:
#initial formating 
print(MGE_Custommgedb_gene_lengths.head())
mge_count_raw.head()

                      Header  Gene Length
0        414_tnpA_AL513383.1         3497
1   2570_tnpA_CCSD01000073.1         3090
2       2004_tnpA_HF677572.1         3084
3  733_tnpA1_CBTR010000003.1         3081
4   1017_tnpA_FKLR01000027.1         3066


,1002_tnpA_JQ364968.1,1004_tnpA_KT779035.1,1017_tnpA_FKLR01000027.1,101_rep20_10_repA(SAP057A)_NC_013334.1,1020_IS91_LFAQ01000019.1,102_rep20_11_repA(VRSAp)_AP003367,1037_tnpA_AY849557.2,103_rep21_1_rep(pWBG754)_GQ900396.1,1041_IS91_AIGD01000018.1,1044_tnpA_AB723628.1,...,954_tnpA_KP987218.1,955_tnpA_JQ318854.1,956_tnpA7_HG916853.1,957_tnpA_KX029332.1,968_tnpA_Z17279.1,978_tnpA_AF141323.1,982_tnpA_JX296013.1,98_rep20_7_repA(SAP074A)_GQ900426.1,997_tnpA_AF342826.1,999_tnpA_rve_CEGG01000033.1
SampleID,,,,,,,,,,,,,,,,,,,,,
IMVM_0001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IMVM_0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IMVM_0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IMVM_0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IMVM_0005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# RPKM Morn
mge_count_rpkm = compute_rpkm(mge_count_raw, MGE_Custommgedb_gene_lengths, gene_col="Header", length_col="Gene Length")
external_arg_count_rpkm.head()

,erm(B)_18_X66468,erm(B)_10_U86375,erm(X)_3_U21300,tet(W)_1_DQ060146,erm(A)_2_AF002716,mef(A)_2_U83667,tet(M)_5_U58985,lsa(C)_1_HM990671,msr(D)_2_AF274302,tet(M)_13_AM990992,cfxA3_1_AF472622,blaTEM-116_1_AY425988,tet(M)_4_X75073,tet(O/W)_5_AM889122,tet(Q)_4_Z21523
SampleID,,,,,,,,,,,,,,,
NMH78,0.0,0.0,0.0,0.0,0.0,0.0,307233.146067,277290.303956,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH64,0.0,0.0,0.0,0.0,0.0,0.0,345423.143351,227713.025535,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH50,0.0,0.0,0.0,0.0,0.0,0.0,353977.518374,216607.954511,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH63,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,676132.521974,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
NMH11,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,536411.495035,0.0,0.0,92748.836152,0.0,0.0,0.0,60774.674518


In [17]:
#Format columns 
mge_count_rpkm_gene = format_columns(mge_count_rpkm, n=1)
print(mge_count_rpkm_gene.columns.tolist()[:10])

# Keep also negative samples for MGEs 
mge_count_rpkm_gene = mge_count_rpkm_gene.reindex(inspire_samples, fill_value=0)
print(mge_count_rpkm_gene.shape)

['Col(BS512)', 'Col(IRGK)', 'Col(MG828)', 'Col(MP18)', 'Col156', 'Col3M', 'Col8282', 'ColE10', 'ColRNAI', 'ColpVC']
(1957, 157)


In [18]:
# Save RPKM tables for downstream analysis
#mge_count_rpkm_gene.to_csv("Norm/MGE_count_RF_RPKM_Gene.csv")
#arg_count_rpkm_gene.to_csv("Norm/ARG_count_RF_RPKM_Gene.csv")
#external_arg_count_rpkm_gene.to_csv("Norm/ExtSet_ARGs_count_RF_RPKM_Gene.csv")

## (2) SparCC analysis  

### Process data prior to analysis

In [19]:
# Keep the 20 most abundant species 
top20_sp = taxonomy_rabun.sum().sort_values(ascending=False).head(20).index.tolist()

# Keep also the 50 most abundant ARGs
top50_args = arg_count_rpkm_gene.sum().sort_values(ascending=False).head(50).index.tolist()

#diplay lists 
print(f'top 20 species: {top20_sp}')
print(f'top 50 ARGs: {top50_args}')

top 20 species: ['Lactobacillus_crispatus', 'Lactobacillus_iners', 'Gardnerella_vaginalis', 'Lactobacillus_jensenii', 'Enterococcus_faecalis', 'Lactobacillus_gasseri', 'Escherichia_coli', 'Bifidobacterium_breve', 'Gardnerella_leopoldii', 'Fannyhessea_vaginae', 'Klebsiella_pneumoniae', 'Staphylococcus_aureus', 'Prevotella_bivia', 'Gardnerella_swidsinskii', 'Gardnerella_piotii', 'Lactobacillus_paragasseri', 'Streptomyces_gancidicus', 'Streptococcus_agalactiae', 'Candida_albicans', 'Prevotella_timonensis']
top 50 ARGs: ['lsa(C)', 'tet(M)', 'erm(B)', 'lsa(A)', "aph(3')-III", 'cfxA3', 'mef(A)', 'msr(D)', 'blaTEM-1B', 'erm(A)', 'tet(Q)', 'erm(X)', 'lnu(C)', 'erm(F)', 'ant(6)-Ia', 'blaTEM-116', "aph(3'')-Ib", 'mre(A)', 'aph(6)-Id', 'fosA', 'sul2', 'mph(C)', 'fosB', 'blaMAL-1', 'tet(W)', 'tet(O)', 'tet(L)', 'tet(A)', 'fosA7', 'erm(C)', 'blaZ', 'cfxA', 'dfrA1', 'dfrA17', 'sul1', 'tet(B)', 'tet(O/32/O)', 'blaTEM-1A', 'OqxA', 'lnu(A)', 'catA2', 'OqxB', 'cfxA4', 'bleO', 'dfrG', 'cat', 'msr(A)', 't

In [20]:
# ORFs belonging to the same Tn916 were grouped prior to MGEs filtering
mge_count_rpkm_sparcc_cleaned = mge_count_rpkm_gene.copy()

cols = [Tn for Tn in mge_count_rpkm_sparcc_cleaned.columns if Tn.startswith("Tn916-")]
mge_count_rpkm_sparcc_cleaned["Tn916"] = mge_count_rpkm_sparcc_cleaned[cols].sum(axis=1)
mge_count_rpkm_sparcc_cleaned.drop(columns=cols, inplace=True)

# Keep also the 50 most abundant MGEs
top50_mges = mge_count_rpkm_sparcc_cleaned.sum().sort_values(ascending=False).head(50).index.tolist()
print (f'top 50 MGEs: {top50_mges}')

top 50 MGEs: ['Tn916', 'tnpA', 'Xis-Tn916', 'ColRNAI', 'IS91', 'Int-Tn916', 'Col156', 'Col(MG828)', 'tnpA1', 'rep6', 'Col(BS512)', 'rep11', 'rep9', 'rep10', 'int2', 'tnpAB', 'IS621', 'tnpA4', 'IncFII(S)', 'tnpAIS1', 'rep21', 'repUS2', 'ColpVC', 'delta-tnpA', 'istB', 'int3', 'IncFIB(AP001918)', 'Col8282', 'rep7', 'repUS11', 'rep20', 'rep18', 'tnpA3', 'rep8', 'IncFIA', 'IncI1', 'rep5', 'tnpA10', 'intI1', 'IncFII(pCRY)', 'tnpA5', 'qacEdelta', 'rep2', 'IncQ1', 'istA2', 'IncFII(29)', 'rep10b', 'tniA', 'IncX1', 'IncFII']


In [21]:
# Formmat table to fit the input requirements of the fastspar tool
# (i) spcies & ARGs 
matrice_top50ARGs = arg_count_rpkm_gene[top50_args]
matrice_top20sp = taxonomy_rabun[top20_sp]
merge_matrice_ARGsp = matrice_top50ARGs.merge(matrice_top20sp, left_index= True, right_index= True)
print(f'merged matrice ARGs+sp shape: {merge_matrice_ARGsp.shape}')

# (ii) species & ARGs & MGEs
matrice_top50MGEs = mge_count_rpkm_sparcc_cleaned[top50_mges]
merge_matrice_ARGspMGE = merge_matrice_ARGsp.merge(matrice_top50MGEs, left_index= True, right_index= True)
print(f'merged matrice ARGs+sp+MGEs shape: {merge_matrice_ARGspMGE.shape}')
merge_matrice_ARGspMGE.head()

merged matrice ARGs+sp shape: (1957, 70)
merged matrice ARGs+sp+MGEs shape: (1957, 120)


,lsa(C),tet(M),erm(B),lsa(A),aph(3')-III,cfxA3,mef(A),msr(D),blaTEM-1B,erm(A),...,tnpA5,qacEdelta,rep2,IncQ1,istA2,IncFII(29),rep10b,tniA,IncX1,IncFII
SampleID,,,,,,,,,,,,,,,,,,,,,
IMVM_0001,0.000000,98329.122646,136932.926518,33139.314918,115559.17641,0.0,6788.390043,3451.383189,0.0,188257.264848,...,0.0,0.0,11367.548245,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0002,676132.521974,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0003,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0004,425963.488844,0.000000,301837.270341,0.000000,0.00000,0.0,0.000000,0.000000,0.0,191256.830601,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IMVM_0005,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Compute a compositionnaly aware correlation matrix using Fastspar (https://github.com/scwatts/fastspar.git)

**(1) Correlation inference**
```{bash} 
fastspar --otu_table merge_matrice_ARGsp.tsv --correlation median_correlation.tsv --covariance median_covariance.tsv
```
**(2) bootstrap**
```{bash} 
mkdir bootstrap_counts
fastspar_bootstrap --otu_table merge_matrice_ARGsp.tsv --number 1000 --prefix bootstrap_counts/merge_matrice_ARGsp
```

**(3) Infer correlation** 
```{bash} 
mkdir bootstrap_correlation
parallel fastspar --otu_table {} --correlation bootstrap_correlation/cor_{/} --covariance bootstrap_correlation/cov_{/} -i 5 ::: bootstrap_counts/*
```
**final p-values table**
```{bash} 
fastspar_pvalues --otu_table merge_matrice_ARGsp.tsv --correlation median_correlation.tsv --prefix bootstrap_correlation/merge_matrice_ARGsp --permutations 1000 --outfile pvalues.tsv
```


### Post-processing of FastSpar outputs

#### load output

In [22]:
# load fastspar output (corr and p-value matrix) (supplementary data tables)

#ARG+sp
SPARGs_median_corr = pd.read_csv("../data/Output/CorAbun_20Sp50ARGs.csv", sep=';', index_col='#OTU ID') 
SPARGs_pvalues_corr = pd.read_csv("../data/Output/CorAbun_20Sp50ARGs_pval.csv", sep=';', index_col='#OTU ID')

#ARG+sp+MGE
SPARGsMGEs_median_corr = pd.read_csv("../data/Output/CorAbun_20Sp50ARG50MGE.csv", sep=';', index_col='#OTU ID') 
SPARGsMGEs_pvalues_corr = pd.read_csv("../data/Output/CorAbun_20Sp50ARG50MGE_pval.csv", sep=';', index_col='#OTU ID')

In [23]:
# Convert a correlation matrix to a df long format for network vizualisation in Gephi
def convert_to_network(corr_matrix, value_name="Weight"):
    network_df = (corr_matrix.stack().reset_index().rename(columns={'#OTU ID': 'Source', 'level_1': 'Target', 0: value_name}))
    network_df = network_df[network_df["Source"] != network_df["Target"]] # Remove self-loops
    # Remove duplicate edges (A–B == B–A)
    network_df["pair"] = network_df.apply(lambda row: tuple(sorted([row["Source"], row["Target"]])), axis=1)
    network_df = network_df.drop_duplicates(subset="pair").drop(columns="pair")

    return network_df

# Filter network dataframe based on correlation and p-value thresholds
def filter_network(network, corr=0.1, pvalue=0.01):
    filter_criteria = (network['Weight'] > corr) & (network['pvalue'] < pvalue)
    network_filtered = network[filter_criteria]
    
    return network_filtered


In [24]:
SPARGs_network_weight = convert_to_network(SPARGs_median_corr, "Weight")
SPARGs_network_pvalue = convert_to_network(SPARGs_pvalues_corr, "pvalue")
SPARGsMGEs_network_weight = convert_to_network(SPARGsMGEs_median_corr, "Weight")
SPARGsMGEs_network_pvalue = convert_to_network(SPARGsMGEs_pvalues_corr, "pvalue")

In [25]:
# spARGs Network (corr > 1 and p-value < 0.01)
network_ARGsp = SPARGs_network_weight.merge(SPARGs_network_pvalue[["Source", "Target", "pvalue"]], on=["Source", "Target"], how="inner")
network_ARGsp_filtered = filter_network(network_ARGsp, corr=0.1, pvalue=0.01)   

# spARGsMGEs Network (corr > 1 and p-value < 0.01)
network_ARGspMGE = SPARGsMGEs_network_weight.merge(SPARGsMGEs_network_pvalue[["Source", "Target", "pvalue"]], on=["Source", "Target"], how="inner")
network_ARGspMGE_filtered = filter_network(network_ARGspMGE, corr=0.1, pvalue=0.01) 

#Print example
network_ARGsp_filtered.head()

# These dfs are used as inputs for Gephi (see Methods for network visualization).

,Source,Target,Weight,pvalue
6,Lactobacillus_crispatus,Lactobacillus_gasseri,0.3786,0.001
7,Lactobacillus_crispatus,Lactobacillus_jensenii,0.4049,0.001
69,Escherichia_coli,Enterococcus_faecalis,0.1900,0.001
72,Escherichia_coli,Citrobacter_koseri,0.4352,0.001
73,Escherichia_coli,Klebsiella_pneumoniae,0.5021,0.001
